# Site-Specific Ligand Binding

In the previous two notebooks we introduced the binary vector notation and hierarchical parameterization.  In this notebook we incorporate these tools into the binding polynomial, a concise mathematical description of a binding model. The binding polynomial is essentially the summation of the concentration of each state, specified in terms of the concentration of the unliganded state (the reference state), the concentration of free ligand, and the thermodynamic parameters (equilibrium constants).

# 3. The Binding Polynomial

The concentration of a liganded state, represented by the binary vector $\mathbf{B}$, is given by:

$$
[M_\mathbf{B}] = [M_\mathbf{0}]\prod_{
\substack{\mathbf{b} \\ 
\forall B_j=0,b_j=0 \\
\forall B_j=1,b_j\in\{0,1\}}}
f(\mathbf{b})
$$

$$ 
f(\mathbf{b}) = \begin{cases}
1 & \sum{b_j}=0 \\
K_\mathbf{b}[L_{i}] & \sum{b_j}=1, b_i=1 \\
\alpha_\mathbf{b} & \sum{b_j}>1
\end{cases}
$$

where $[M_\mathbf{0}]$ is the concentration of the unliganded state, $[L_i]$ is the concentration of free ligand for site $i$, and $K_\mathbf{b}$ and $\alpha_\mathbf{b}$ are the equilibrium constants. Symmetry between the binding sites, e.g. for cyclic protein oligomers, can be readily incorporated using the binary notation, resulting in equivalenced states and a simplified model. 

We generate the bind polynomial based on the equations above, implemented in the python scripts associated with this package.

### This workbook demonstrates how to derive a highly customized model

First we initialize the model by assigning the number of sites and import some functions that will be useful later. Note that once an assumption has been applied it cannot (easily) be reversed and it is best restart the notebook if you want to change the assumption.

**Initialise each code block (`In [ ]:`) by clicking on the cell and pressing `SHIFT`+`ENTER`.**

As a simple example let us first consider a cyclically symmetric heptamer (`n_sites=7`, `sym='cyclic`). Note that for dih we take the first.

In [1]:
from site_specific_model import SiteSpecificModel
from sympy import expand,symbols, init_printing

mymodel = SiteSpecificModel(n_sites=7,sym='cyclic')

Let us assume that only nearest neighbors interact in a non-additive fashion.

In [2]:
# apply simple nearest neighbor approximations
mymodel.nearest_neighbors()
# perform the computations to construct the binding polynomial
mymodel.generate_bp()
# print the binding polynomial
mymodel.bp

1 + 7*[L_{1}]*K_{1000000} + 14*[L_{1}]**2*K_{1000000}**2 + 7*[L_{1}]**3*K_{1000000}**3 + 7*\alpha_{1100000}*[L_{1}]**2*K_{1000000}**2 + 21*\alpha_{1100000}*[L_{1}]**3*K_{1000000}**3 + 7*\alpha_{1100000}*[L_{1}]**4*K_{1000000}**4 + 7*\alpha_{1100000}**2*[L_{1}]**3*K_{1000000}**3 + 21*\alpha_{1100000}**2*[L_{1}]**4*K_{1000000}**4 + 7*\alpha_{1100000}**3*[L_{1}]**4*K_{1000000}**4 + 14*\alpha_{1100000}**3*[L_{1}]**5*K_{1000000}**5 + 7*\alpha_{1100000}**4*[L_{1}]**5*K_{1000000}**5 + 7*\alpha_{1100000}**5*[L_{1}]**6*K_{1000000}**6 + \alpha_{1100000}**7*[L_{1}]**7*K_{1000000}**7

We can also print the binding polynomial a bit nicer.

In [3]:
init_printing()

display(expand(mymodel.bp))

### Another example

Let us consider a more complex example. Here we specify 5 ligands (`n_sites=5`), with no symmetry between the sites (`sym='none'`). 

In [4]:
mymodel = SiteSpecificModel(n_sites=5,homotropic=False,sym='none')

Even though we did not assume the sites bind the same ligand (`homotropic=False`), we can customise the model such a some sites bind the same ligand. We do this by populating the `L` list variable of the model with sympy symbols representing the free concentration of the different ligands.

In the example below Site 1 binds one ligand and Sites 2-5 bind a different ligand.

In [5]:
L1,L2 = symbols('[L_{1}] [L_{2}]')
mymodel.L = [L1,L2,L2,L2,L2]

Let us further assume that the Sites 2-5 are equivalent (for example they may be identical domains). Note that we are not assuming the interactions between them are identical.

To do this we use the `set_equivalent_affinities` function of the model and provide it with a list of the sites to be assigned as equivalent.

In [6]:
mymodel.set_equivalent_affinities([2,3,4,5])

Now let us make some assumptions about the interactions between the sites. We can make the interactions between some sites negligible using the `set_independent` function of the model or make the interactions extremely unfavorable by using the `set_block` function.

In the example below we assume all of the interactions between 3 or more sites are negligible and only single ligand 2 can bind at one time by setting the interactions between the sites binding ligand 2 as extremely unfavorable. The latter may occur if ligand 2 is very large.

In [7]:
for s in mymodel.states:
    # if a state contains 3 or more sites occupied, set the relevant interaction constant to 1
    if(s.count('1')>=3):
        mymodel.set_independent(s)
# set the interaction constant between these pairs of sites to 0
mymodel.set_block(['01100','00110','00011'])

Finally, we assume that Site 1 interacts in the same way with Sites 2-5. We introduce this approximation by modifying the `params` dictionary variable of the model. Note that we have used the same interactions as above.

In [8]:
# define the states to be made equivalent
equiv_interactions = ['11000','10100','10010','10001']

for s in mymodel.params:
    if(s in equiv_interactions[1:]):
        mymodel.params[s] = mymodel.params[equiv_interactions[0]]

Now that we have made our approximations, we are ready to make the binding polynomial that summarizes our model. Note for models with >7 sites, applying simplifications before generating the binding polynomial speeds up its generation considerably. 

In [9]:
# perform the computations to construct the binding polynomial
mymodel.generate_bp()

# print the binding polynomial in a nice format
display(expand(mymodel.bp))

As a last step let us list all the fittable parameters in our model.

In [10]:
mymodel.make_params_list()

# print each parameter in a nice format
for param in mymodel.params_list:
    display(expand(param))

We have dropped from the 31 ($=2^5-1$) parameters of the general 5-site model, to only 6 free parameters.

### Final example

In the last example, we consider a dihedral dodecamer (`n_sites=12`). Here the C2 symmetry axis is represented by grouping the first 6 binary strings and the last 6 binary strings such that e.g. `100000 = 0001000`).

In [11]:
mymodel = SiteSpecificModel(n_sites=12,sym='dihedral')

Now we simplify this model by using a nearest neighbor approximation in which adjacent subunits along the C6 symmetry axis can interact and one set of subunits along the C2 symmetry axis can interact.

In [12]:
for s in mymodel.states:
    # if a state contains 3 or more sites occupied, set the relevant interaction constant to 1
    if(s.count('1')>=3):
        mymodel.set_independent(s)
    elif(s.count('1')==2):
        if(s=='110000000000' or s=='100000100000'):
            pass
        else:
            mymodel.set_independent(s)

The binding polynomial "reduces" to:

In [13]:
# perform the computations to construct the binding polynomial
mymodel.generate_bp()

# print the binding polynomial in a nice format
display(expand(mymodel.bp))

Which seems complex but only has 3 model parameters, a massive reduction from the 143 parameters of the 12-site general model.